# 参考資料

[torchtextで簡単にDeepな自然言語処理](https://qiita.com/itok_msi/items/1f3746f7e89a19dafac5)

[自分のデータでサクッとattentionつき文書分類モデルをためす.ipynb](https://github.com/nn116003/torchtext-tutorial/blob/master/%E8%87%AA%E5%88%86%E3%81%AE%E3%83%87%E3%83%BC%E3%82%BF%E3%81%A7%E3%82%B5%E3%82%AF%E3%83%83%E3%81%A8attention%E3%81%A4%E3%81%8D%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E%E3%83%A2%E3%83%87%E3%83%AB%E3%82%92%E3%81%9F%E3%82%81%E3%81%99.ipynb)

[LANGUAGE TRANSLATION WITH TORCHTEXTT](https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html)


In [1]:
print('hello world')

hello world


# モデルの定義


In [2]:
from google.colab import drive
drive.mount('/content/dirve')

Mounted at /content/dirve


In [3]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor


class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden


class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)


class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs

# データセットの前処理


In [5]:
!pip install janome
!pip install torchtext==0.6.0

     |████████████████████████████████| 19.7MB 49.5MB/s 
     |████████████████████████████████| 71kB 2.1MB/s 
     |████████████████████████████████| 1.1MB 6.9MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [6]:
import janome
from janome.tokenizer import Tokenizer
import torchtext
import torch
import spacy
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe

In [7]:
from janome.tokenizer import Tokenizer
j_t = Tokenizer()
def tokenizer(text): 
    return [tok for tok in j_t.tokenize(text, wakati=True)]

In [8]:
SRC = data.Field(sequential=True, tokenize=tokenizer,init_token='<sos>',
                            eos_token='<eos>', lower=True)
TRG = data.Field(sequential=True, tokenize=tokenizer,init_token='<sos>',
                            eos_token='<eos>', lower=True)

In [9]:
train, val, test = data.TabularDataset.splits(
        path="/content/dirve/My Drive/Colab Notebooks/", train='train.tsv',
        validation='val.tsv', test='test.tsv', format='tsv',
        fields=[('SRC', SRC), ('TRG', TRG)])

In [10]:
 print('len(train)', len(train))
 print('vars(train[0])', vars(train[0]))

len(train) 20000
vars(train[0]) {'SRC': ['16', '年間', 'その', '先生', 'に', 'お世話', 'に', 'なっ', 'て'], 'TRG': ['長い', 'です', 'ね']}


In [11]:
SRC.build_vocab(train, min_freq=1)
TRG.build_vocab(train, min_freq=1)

In [12]:
len(TRG.vocab)

2349

In [13]:
"""
from torchtext.vocab import FastText
SRC.build_vocab(train, vectors=FastText(language="ja"), min_freq=2)
TRG.build_vocab(train, vectors=FastText(language="ja"), min_freq=2)
"""

'\nfrom torchtext.vocab import FastText\nSRC.build_vocab(train, vectors=FastText(language="ja"), min_freq=2)\nTRG.build_vocab(train, vectors=FastText(language="ja"), min_freq=2)\n'

In [14]:
train_batch_size = 50
test_batch_size = 10
eval_batch_size = 2
train_iter, val_iter, test_iter = data.BucketIterator.splits((train, val, test), sort = False,  batch_sizes = (train_batch_size,eval_batch_size, test_batch_size), device= device)

In [15]:
#TEXT.vocab.stoi

In [16]:
# バッチ毎に大きさが違うのでエラーを吐く 
#　解決しました
#for _, batch in enumerate(train_iter):
  #print("input size: ", batch.SRC.shape)
  #print("output: ", batch.OUT)

# インスタンスの初期化


In [17]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [18]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

In [19]:
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(3652, 32)
    (rnn): GRU(32, 64, bidirectional=True)
    (fc): Linear(in_features=128, out_features=64, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=192, out_features=8, bias=True)
    )
    (embedding): Embedding(2349, 32)
    (rnn): GRU(160, 64)
    (out): Linear(in_features=224, out_features=2349, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [20]:
optimizer = optim.Adam(model.parameters())

In [21]:
def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 811,381 trainable parameters


In [22]:
PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [27]:
import math
import time


def train(model: nn.Module,
          iterator: data.BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0
    #print(len(iterator))

    for _, batch in enumerate(iterator):

        src = batch.SRC
        trg = batch.TRG

        optimizer.zero_grad()

        output = model(src, trg)
        print(output.shape)
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        print(output.shape)
        print(trg.shape)
        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [24]:
def evaluate(model: nn.Module,
             iterator: data.BucketIterator,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, batch in enumerate(iterator):

            src = batch.SRC
            trg = batch.TRG

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


In [25]:
def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [28]:
N_EPOCHS = 1
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

torch.Size([12, 50, 2349])
torch.Size([550, 2349])
torch.Size([550])
torch.Size([15, 50, 2349])
torch.Size([700, 2349])
torch.Size([700])
torch.Size([13, 50, 2349])
torch.Size([600, 2349])
torch.Size([600])
torch.Size([13, 50, 2349])
torch.Size([600, 2349])
torch.Size([600])
torch.Size([10, 50, 2349])
torch.Size([450, 2349])
torch.Size([450])
torch.Size([10, 50, 2349])
torch.Size([450, 2349])
torch.Size([450])
torch.Size([13, 50, 2349])
torch.Size([600, 2349])
torch.Size([600])
torch.Size([15, 50, 2349])
torch.Size([700, 2349])
torch.Size([700])
torch.Size([11, 50, 2349])
torch.Size([500, 2349])
torch.Size([500])
torch.Size([11, 50, 2349])
torch.Size([500, 2349])
torch.Size([500])
torch.Size([9, 50, 2349])
torch.Size([400, 2349])
torch.Size([400])
torch.Size([9, 50, 2349])
torch.Size([400, 2349])
torch.Size([400])
torch.Size([9, 50, 2349])
torch.Size([400, 2349])
torch.Size([400])
torch.Size([12, 50, 2349])
torch.Size([550, 2349])
torch.Size([550])
torch.Size([10, 50, 2349])
torch.Size

KeyboardInterrupt: ignored

# 文章の生成用の関数

参考：[pytorch-seq2seq/6 - Attention is All You Need.ipynb](https://github.com/bentrevett/pytorch-seq2seq/blob/master/6%20-%20Attention%20is%20All%20You%20Need.ipynb)

[CookieBox26/20200613.ipynb](https://gist.github.com/CookieBox26/2dec17af9dcf6d22848b15d3226282ea)



In [ ]:
def gen_sentence(sentence, src_field, trg_field, model, batch_size):
  model.eval()
  in_str, out_str, pred = [], [], []
  length = len(sentence)

  with torch.no_grad():
    for _, batch in enumerate(sentence):
      src = batch.SRC
      trg = batch.TRG

      output = model(src, trg)
          
      for j in range(min(length, batch_size)):
        topv, topi = output.data.topk(1)
        _, topi_s = output.data.topk(2) 
        for k in range(topi.size()[1]):
          if topi[1:, k][0] == trg_field.vocab.stoi["<eos>"]:
            for i in range(topi_s.size()[0]):
              for l in range(topi_s.size()[1]):
                topi[i][l][0] = topi_s[i][l][1]
          pred.append([trg_field.vocab.itos[topi[1:, k][i]] for i in range(topi.size()[0] - 1)])
        in_str.append([src_field.vocab.itos[i.item()] for i in src[:,j]])
        out_str.append([trg_field.vocab.itos[i.item()] for i in trg[:,j]])
      
  return in_str, out_str, pred


In [ ]:
# 中間発表時にはテストデータは用いない
test_in, test_out, test_pred = [],[],[]
test_in, test_out, test_pred = gen_sentence(test_iter, SRC, TRG, model, test_batch_size)

In [ ]:
val_in, val_out, val_pred = [],[],[]
val_in, val_out, val_pred = gen_sentence(val_iter, SRC, TRG, model, eval_batch_size)

In [ ]:
train_in, train_out, train_pred = [],[],[]
train_in, train_out, train_pred = gen_sentence(train_iter, SRC, TRG, model, train_batch_size)

In [ ]:
import pandas as pd

In [ ]:
def convert_list_to_df(in_list, out_list, pred_list):
  row = []
  for i in range(len(in_list)):
    batch_input = in_list[i]
    batch_output = out_list[i]
    batch_pred = pred_list[i]
    input = [j for j in batch_input if j != "<pad>" and j != "<sos>" and j != "<eos>" and j != "<unk>"]
    output = [j for j in batch_output if j != "<pad>" and j != "<sos>" and j != "<eos>" and j != "<unk>"]
    predict = [j for j in batch_pred if j != "<pad>" and j != "<sos>" and j != "<eos>" and j != "<unk>"]
    input_str = "".join(input)
    output_str ="".join(output)
    predict_str = "".join(predict)
    row.append([input_str, output_str, predict_str])

  df = pd.DataFrame(row, columns=["input","answer","predict"])
  df = df.sort_values('input')
  return df

In [ ]:
train_df = convert_list_to_df(train_in, train_out, train_pred)
val_df = convert_list_to_df(val_in, val_out, val_pred)
test_df = convert_list_to_df(test_in, test_out, test_pred)

In [ ]:
df_s = pd.concat([train_df, test_df])

In [ ]:
df_s = df_s.sort_values("input")

In [ ]:
df_s.to_csv("/content/dirve/My Drive/Colab Notebooks/result_transformer.csv")